# Sketching the pig reads: Something went wrong
- Sketched read1 2x, instead of sketching read1 and read2
- Can this explain the differences in RGI, in viral too? 

This was not true, sketches were good. 
Manysketch seems to be easiest. 







In [ ]:
rule manysketch_csv:
    input:
        read_one=f"{ATLAS_DIR}/atlas_{{metag}}/{{metag}}/sequence_quality_control/{{metag}}_QC_R1.fastq.gz",
        read_two=f"{ATLAS_DIR}/atlas_{{metag}}/{{metag}}/sequence_quality_control/{{metag}}_QC_R2.fastq.gz",
    output:
        csv = "../results/metag_sig/manysketch_csv/{metag}.csv",
    conda: 
        "branchwater-skipmer"
    threads: 1
    shell: 
        """
        echo name,name,read1,read2 > manysketch.csv
        echo {wildcards.metag},{input.read1},{input.read2} >> {output.csv} 
        """


In [ ]:
echo name,name,read1,read2 > manysketch.csv
for i in *.fa.gz
do
echo $i,$i,
done >> manysketch.csv

Pretty sure i did it right tho? In the snakefile?
acc
SRR14369135
SRR15057928
SRR5240728
SRR8960303
SRR8960562

In [ ]:
srun --account=ctbrowngrp -p med2 -J sketch -t 48:00:00 -c 40 --mem=100gb --pty bash
# make one merged sketch for all vOTU 

mamba activate branchwater-skipmer

snakemake -s sketch.smk --use-conda --resources mem_mb=100000 --rerun-triggers mtime \
-c 40 --rerun-incomplete -k -n

In [ ]:
srun --account=ctbrowngrp -p med2 -J smash -t 1:00:00 -c 14 --mem=30gb --pty bash

module load parallel
mamba activate branchwater-skipmer 
for f in *.fasta
do
sourmash sketch dna \
-p k=21,scaled=1000,k=31,scaled=1000,k=51,scaled=1000 \
$f --name ${f%.fasta*} -o \
../../../../../sourmash/sig_files/MAGs2.0/${f%.fasta*}.zip
done | parallel -j 32